In [14]:
%pip install tensorflow os pillow

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)

2024-04-05 15:37:12.105087: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 15:37:12.109335: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 15:37:12.165969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 15:37:13.100576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
import os

#idea: loop through each file in each subfolder, save the image in a list e.g. 'list_of_images'
#in the same loop save a list of the label for each image e.g. 'list_of_labels'
        #e.g. by extracting the string before / e.g. 'email' (using append method)
#the list should be of same lengts, merge in the end 


filepath = os.path.join("..",
                        "..",
                        "..",
                        "..",
                        "..",
                        "cds-vis-data",
                        "Tobacco3482")

def load_file_label
        for file in sorted(os.listdir(filepath)):

                #initialize 2 empty lists
                list_of_images= []
                list_of_labels=[]


                individual_filepath= os.path.join(filepath, file)
                image = load_img(individual_filepath, target_size=(224, 224)) # Read in file
                

                #append image to list_of_images
                list_of_images=list_of_images.append(image)

                #extract the label for each image
                #in individual_filepath extract what is between / and / e.g. /News/
                #label= 

                #append label to list_of_labels
                list_of_labels=list_of_labels.append(label)

                individual_filepath = file.split(".")[0] 
        return list_of_images, list_of_labels

#now merge the 2 lists 

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.